# DeepMReyeCalib


#### Eye movement data analysis:

- [x] Step 1. Extract time series
- [X] Step 2. Extract saccades for each trials

#### Step 1: extract time series

In [1]:
# Imports
import os
import numpy as np
import glob
import pandas as pd
from mat4py import loadmat
from sac_utils import vecvel, microsacc_merge, saccpar, isincircle
import ipdb
import math 

from gaze_utils import *

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template
import plotly.io as pio

/var/folders/v8/zmb9hj_97tjcq8mb570v8wq40000gp/T/ipykernel_15154/803841803.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Define parameters
num_trials = 118                      # number of trials per run
#parameters for model (saccade detection)
sampling_rate = 1000                   # eyetrakcing sampling rate
velocity_th = 1.5                      # velocity sd threshold
min_dur = 20                           # threshold minimum duration
merge_interval = 20                    # interval between saccade events
cor_sac_onset_th = 100                 # corrective saccade onset threshold (inferior or equal in ms)
fix_trial_end_soa = 800                # duration between fixation offset and trial end for blink checking 
fix_area_rad = 2                       # saccade onset position tolerance area in dva
sac_area_rad = 3                       # saccade landing area tolerance in dva
sac_lat_min = 50                       # saccade latency minimum duration
sac_lat_max = 400                      # saccade latency maximum duration

In [3]:
# Define folders
# when on mac
base_dir = r'/Users/sinakling/projects/deepmreyecalib/experiment_code' 
data_dir = '{}/data'.format(base_dir)
subject = 'sub-01'

subject_num = subject[4:]
fig_dir = '{data_dir}/{subject}/figures'.format(data_dir=data_dir, subject=subject)
training = 0

In [5]:
# Define data filenames
if training == 1:
    data_events = sorted(glob.glob(r'{}/{}/ses-01/beh/*.tsv'.format(data_dir,subject)))
    num_run = len(data_events)

    data_eyetrack = sorted(glob.glob(r'{}/{}/ses-01/beh/*.edf'.format(data_dir,subject)))
    data_mat = sorted(glob.glob(r'{}/{}/ses-01/beh/*.mat'.format(data_dir,subject)))
else: 
    data_events = sorted(glob.glob(r'{}/{}/ses-01/func/*.tsv'.format(data_dir,subject)))
    num_run = len(data_events)

    data_eyetrack = sorted(glob.glob(r'{}/{}/ses-01/func/*.edf'.format(data_dir,subject)))
    data_mat = sorted(glob.glob(r'{}/{}/ses-01/func/*.mat'.format(data_dir,subject)))



assert len(data_eyetrack) > 0, "No eyetracking data found"
assert len(data_mat) > 0, "No matlab data found"
assert len(data_events) > 0, "No event files found"

In [ ]:
# Create message and data files (run only once first time)
for run in data_eyetrack:
    
    if not os.path.exists(run.replace('.edf','.msg')):
        #os.system('edf2asc.exe {} -e -y'.format(run))    #! windows EXECUTE through cmd (start edf2asc "/Users/sinakling/projects/deepmreyecalib/data/sub-01/ses-01/beh/sub-01_ses-01_task-DeepMReyeCalib_training_run-01_eyetrack.edf" -e -y)
        os.rename(run.replace('.edf','.asc'),run.replace('.edf','.msg'))    

#### Checking msg files: 

Correct timing (see later)

- timestamps of eventfile are in seconds: (last timestamp - first timestamp) / 60 
- timestamps of eyetracker file (msg) are in milliseconds: (last trial timestamp - first trial timestamp) / 60000

MRI trigger 

- should be printed in msg at trial 1


In [ ]:
# Create dat files (run only once first time)
for run in data_eyetrack: 
    if not os.path.exists(run.replace('.edf','.dat')): 
       # os.system('edf2asc.exe {} -s -miss -1.0 -y'.format(run)) #!EXECUTE through cmd  (start edf2asc "/Users/sinakling/projects/deepmreyecalib/data/sub-01/ses-01/beh/sub-01_ses-01_task-DeepMReyeCalib_training_run-01_eyetrack.edf" -s -miss -1.0 -y)
       os.rename(run.replace('.edf','.asc'),run.replace('.edf','.dat'))

In [ ]:
# Collect MSG data
# saves timestamps from messages send to eyetracking data in dataframe: one per run, one for all runs together per subject


#adapted for DeepMReyeCalib 

msg_outputs = ['trial_onset', 'trial_offset', 'check_fix_onset', 'iti_onset', 'iti_offset', 
               'fixation_onset', 'fixation_offset', 'pursuit_onset', 'pursuit_offset', 'freeview_onset', 
               'freeview_offset']
 
for msg_output in msg_outputs:
    exec("{} = np.zeros(num_trials*num_run)".format(msg_output))

t_run = 0
dfs_list = []

record_lines = []

for run in data_eyetrack:
    
    msgfid = open(run.replace('.edf','.msg'))
    first_last_time, first_time, last_time = False, False, False

    while not first_last_time:
        line_read = msgfid.readline()

        if not line_read == '':
            la = line_read.split()
    
            if len(la) > 2:
                if la[2] == 'RECORD_START' and not first_time:                      #Eyelink('message', 'RECORD_START'); 
                    first_time = True
                    record_lines.append(line_read)
                if la[2] == 'RECORD_STOP' and not last_time:                        #Eyelink('message', 'RECORD_STOP');
                    last_time = True
                    record_lines.append(line_read)
            if len(la) > 4:
                if la[2] == 'trial' and la[4]=='started':                           #Eyelink('message', '%s', sprintf('trial %i started\n', t));
                    trial_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])   
                if la[2] == 'trial' and la[4]=='ended':                             #Eyelink('message', '%s', sprintf('trial %i ended\n', t));
                    trial_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'iti' and la[4]=='onset':                               #Eyelink('message,'%s',log_txt); log_txt = sprintf('iti %i onset at %f', t, vbl);
                    iti_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'iti' and la[4]=='offset':
                    iti_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'fixation' and la[4]=='onset':                          #Eyelink('message,'%s',log_txt); log_txt = sprintf('fixation %i onset at %f', t, vbl);
                    fixation_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'fixation' and la[4]=='offset':
                    fixation_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'pursuit' and la[4]=='onset':
                    pursuit_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1]) #Eyelink('message,'%s',log_txt); log_txt = sprintf('pursuit %i onset at %f', t, vbl);
                if la[2] == 'pursuit' and la[4]=='offset':
                    pursuit_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'freeview' and la[4]=='onset':
                    freeview_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1]) #Eyelink('message,'%s',log_txt); log_txt = sprintf('freeview %i onset at %f', t, vbl);
                if la[2] == 'freeview' and la[4]=='offset':
                    freeview_offset[int(la[3])-1 + (t_run)*num_trials,] = float(la[1])
    
        if first_time and last_time:
            first_last_time = True
            msgfid.close();
    t_run += 1

# create events dataframe
for run_num, run in enumerate(data_events):
    df_run = pd.read_csv(run, sep="\t")
    if run_num  > 0 :
        df_events = pd.concat([df_events, df_run])
    else :
        df_events = df_run
        

msg_dict = {}
for msg_output in msg_outputs:
    eval("msg_dict.update({'%s':%s})"%(msg_output,msg_output))
    
msg_dict.update({'iti_duration': iti_offset-iti_onset})
msg_dict.update({'fixation_duration': fixation_offset-fixation_onset})
msg_dict.update({'pursuit_duration': pursuit_offset-pursuit_onset})
msg_dict.update({'freeview_duration': freeview_offset-freeview_onset})

df_msg = pd.DataFrame(msg_dict)
df_all = pd.concat([df_events.reset_index(drop=True),
                    df_msg.reset_index(drop=True)], axis=1)

# Save DataFrame for each run
num_trials_per_run = int(len(df_all) / num_run)
for i in range(num_run):
    start_idx = i * num_trials_per_run
    end_idx = (i + 1) * num_trials_per_run if i < (num_run - 1) else len(df_all)
    df_run_single = df_all.iloc[start_idx:end_idx]
    df_run_single.to_csv(f'{subject}_run_{i + 1}_messages.csv', index=False)

# Append DataFrame to the list
dfs_list.append(df_all)

t_run += 1

# Concatenate all DataFrames from the list into a final DataFrame
final_df = pd.concat(dfs_list, ignore_index=True)

# Save the final concatenated DataFrame
final_df.to_csv(f'{subject}_all_runs_messages.csv', index=False)


In [ ]:
# Extract recording times from msg file 

record_dict = {}
record_start_count = 1
record_stop_count = 1

for line in record_lines:
    parts = line.split()
    timestamp = int(parts[1])
    action = parts[2]

    if action == 'RECORD_START':
        key = f'RECORD_START_{record_start_count}'
        record_dict[key] = timestamp
        record_start_count += 1
    elif action == 'RECORD_STOP':
        key = f'RECORD_STOP_{record_stop_count}'
        record_dict[key] = timestamp
        record_stop_count += 1

print(record_dict)

In [ ]:
# Read message csvs
df_run_1 =  pd.read_csv(f'{subject}_run_1_messages.csv')
df_run_2 =  pd.read_csv(f'{subject}_run_2_messages.csv')
df_run_3 =  pd.read_csv(f'{subject}_run_3_messages.csv')

dfs_runs = [df_run_1,df_run_2, df_run_3]

df_run_1

In [ ]:
# compute time instruction reading time in beginn# compute time instruction reading time in beginning 

# Print timing check statements

instruction_times = []

for run_num,run in zip(range(len(dfs_runs)), dfs_runs):
    duration = int(run['trial_onset'][0]) - record_dict.get(f'RECORD_START_{run_num + 1}')
    instruction_times.append(duration) 
    print(f"Duration of experiment in run {run_num + 1} was {(run['trial_offset'][117] - run['trial_onset'][0])/60000} minutes")
    print(f"Duration of eyelink recording in run {run_num + 1} was {(record_dict.get(f'RECORD_STOP_{run_num + 1}')- record_dict.get(f'RECORD_START_{run_num + 1}'))/60000} minutes")

#add 500ms for plot accuracy
instruction_times = [x + 500 for x in instruction_times]

print(instruction_times)

ending_times = []

for run_num,run in zip(range(len(dfs_runs)), dfs_runs):
    duration = int(run["trial_offset"][117]) - record_dict.get(f'RECORD_STOP_{run_num + 1}')
    ending_times.append(duration) 

print(ending_times)

Step 2: Extract Gaze Positions 

-to plot: center each value (-middle of screen x and y respecitively), convert to degrees 

In [ ]:
# Load eyetracking data 
# onsets and offsets per task 
# Saves csv per run

# Initialize dictionaries to store positions for each task
positions_per_task = {task: [] for task in ['iti', 'fixation', 'pursuit', 'freeview']}

# Iterate over eyetracking data files
for run, num in zip(data_eyetrack,range(len(dfs_runs))):
    print(num)
    for task in positions_per_task.keys():
        # Load eyetracking data
        eye_data_run = np.genfromtxt(run.replace('.edf', '.dat'), usecols=(0, 1, 2))

        np.savetxt(f'eye_data_{subject}_run_{num + 1}.csv', eye_data_run, delimiter=',')

        # Extract x and y positions for onsets and offsets of each trial
        for _, trial_info in dfs_runs[num].iterrows():
            task_onset_col = f"{task}_onset"
            task_offset_col = f"{task}_offset"
            trial_data_logic = np.logical_and(eye_data_run[:, 0] >= float(trial_info[task_onset_col]),
                                            eye_data_run[:, 0] <= float(trial_info[task_offset_col]))

            if np.any(trial_data_logic):
                onset_x = eye_data_run[trial_data_logic][0, 1]  # x position at onset
                onset_y = eye_data_run[trial_data_logic][0, 2]  # y position at onset
                offset_x = eye_data_run[trial_data_logic][-1, 1]  # x position at offset
                offset_y = eye_data_run[trial_data_logic][-1, 2]  # y position at offset

                # Append positions to the dictionary
                trial_positions = np.array([onset_x, onset_y, offset_x, offset_y])
                positions_per_task[task].append(trial_positions)

                
            else:
                # Append NaN if no data points are found
                positions_per_task[task].append(np.array([np.nan, np.nan, np.nan, np.nan]))
                print(f"No data points found for Run {trial_info['run_number']}, Trial {trial_info['trial_number']}")

# Convert the dictionary of positions to a dictionary of NumPy arrays
positions_per_task = {task: np.array(positions) for task, positions in positions_per_task.items()}

# Now you have separate arrays for each task
for task, positions in positions_per_task.items():
    print(f"All Positions for {task} task:")
    print(positions)



In [ ]:
# Remove lines with BLINK message

# Define the number of runs
num_runs = 3

for run_num, data in zip(range(1, num_runs + 1), data_eyetrack):
    # Load eye tracking data for each run
    eye_data_run = np.genfromtxt(data.replace('.edf', '.dat'), usecols=(0, 1, 2))

    # Initialize variables for each run
    eye_data_runs = eye_data_run
    blinkNum = 0
    blink_start = False

    for tTime in np.arange(0, eye_data_runs.shape[0], 1):
        if not blink_start:
            if eye_data_runs[tTime, 1] == -1:
                blinkNum += 1
                timeBlinkOnset = eye_data_runs[tTime, 0]
                blink_start = True
                if blinkNum == 1:
                    blink_onset_offset = np.matrix([timeBlinkOnset, np.nan])
                else:
                    blink_onset_offset = np.vstack((blink_onset_offset, [timeBlinkOnset, np.nan]))

        if blink_start:
            if eye_data_runs[tTime, 1] != -1:
                timeBlinkOffset = eye_data_runs[tTime, 0]
                blink_start = 0
                blink_onset_offset[blinkNum - 1, 1] = timeBlinkOffset

    # Nan record around detected blinks
    eye_data_runs_nan_blink = np.copy(eye_data_runs)

    # Extend time range
    time_range_extension = 200

    for tBlink in np.arange(0, blinkNum, 1):
        onset_time = blink_onset_offset[tBlink, 0]
        offset_time = blink_onset_offset[tBlink, 1]

        # Define the extended time range
        extended_onset = onset_time - time_range_extension
        extended_offset = offset_time + time_range_extension

        # Set values in the specified time range to NaN
        eye_data_runs_nan_blink[
            np.logical_and(
                eye_data_runs_nan_blink[:, 0] >= extended_onset,
                eye_data_runs_nan_blink[:, 0] <= extended_offset,
            ),
            1,
        ] = np.nan

        eye_data_runs_nan_blink[
            np.logical_and(
                eye_data_runs_nan_blink[:, 0] >= extended_onset,
                eye_data_runs_nan_blink[:, 0] <= extended_offset,
            ),
            2,
        ] = np.nan

    # Save the modified data for each run
    output_file_path = f"eye_data_{subject}_run_{run_num}_nan_blink.csv"
    np.savetxt(output_file_path, eye_data_runs_nan_blink, delimiter=",")


In [6]:
import scipy.io

matfile = scipy.io.loadmat(data_mat[0])
scr_sizeX = matfile['config']['scr'][0,0]['scr_sizeX'][0][0][0][0]
scr_sizeY = matfile['config']['scr'][0,0]['scr_sizeY'][0][0][0][0]
screen_size = np.array([scr_sizeX,scr_sizeY])
ppd = matfile['config']['const'][0,0]['ppd'][0][0][0][0]


In [7]:
print(screen_size)
print(ppd)

[1920 1080]
52.7161298750632


In [ ]:
# Load eyedata per run as np array
# Center and convert to dva
eye_data_run_1 = pd.read_csv(f"eye_data_{subject}_run_1_nan_blink.csv")
eye_data_run_1 = np.array(eye_data_run_1)

x_dva_run_1 = (eye_data_run_1[:,1] - (screen_size[0]/2))/ppd
y_dva_run_1 =  -1.0*((eye_data_run_1[:,2] - (screen_size[1]/2))/ppd)



eye_data_run_2 = pd.read_csv(f"eye_data_{subject}_run_2_nan_blink.csv")
eye_data_run_2 = np.array(eye_data_run_2)

x_dva_run_2 = (eye_data_run_2[:,1] - (screen_size[0]/2))/ppd
y_dva_run_2 = -1.0*((eye_data_run_2[:,2] - (screen_size[1]/2))/ppd)



eye_data_run_3 = pd.read_csv(f"eye_data_{subject}_run_3_nan_blink.csv")
eye_data_run_3 = np.array(eye_data_run_3)

x_dva_run_3 = (eye_data_run_3[:,1] - (screen_size[0]/2))/ppd
y_dva_run_3 = -1.0*((eye_data_run_3[:,2] - (screen_size[1]/2))/ppd)


eye_data_all_runs = [eye_data_run_1,eye_data_run_2,eye_data_run_2]



In [ ]:
eyetracking_data_all_runs_x = [x_dva_run_1[instruction_times[0]:ending_times[0]],x_dva_run_2[instruction_times[1]:ending_times[1]],x_dva_run_3[instruction_times[2]:ending_times[2]]]
eyetracking_data_all_runs_y = [y_dva_run_1[instruction_times[0]:ending_times[0]],y_dva_run_2[instruction_times[1]:ending_times[1]],y_dva_run_3[instruction_times[2]:ending_times[2]]]

In [ ]:
# Interpolate blink periods for plots 

def interpol_nans(eyetracking_data_list):
    interpolated_data_list = []
    for i in range(len(eyetracking_data_list)):
        eyetracking_no_nans = np.nan_to_num(eyetracking_data_list[i])
        nan_indices = np.isnan(eyetracking_data_list[i])

        # Interpolate NaN values using linear interpolation
        eyetracking_signal_interpolated = np.interp(np.arange(len(eyetracking_data_list[i])), np.where(~nan_indices)[0], eyetracking_no_nans[~nan_indices])
        interpolated_data_list.append(eyetracking_signal_interpolated)

    return interpolated_data_list


In [ ]:
eyetracking_data_all_runs_x_intpol = interpol_nans(eyetracking_data_all_runs_x)
eyetracking_data_all_runs_y_intpol = interpol_nans(eyetracking_data_all_runs_y)

In [ ]:
np.save(f"eyetracking_x_data_Calib_{subject}_run_1", eyetracking_data_all_runs_x_intpol[0])
np.save(f"eyetracking_y_data_Calib_{subject}_run_1", eyetracking_data_all_runs_y_intpol[0])

np.save(f"eyetracking_x_data_Calib_{subject}_run_2", eyetracking_data_all_runs_x_intpol[1])
np.save(f"eyetracking_y_data_Calib_{subject}_run_2", eyetracking_data_all_runs_y_intpol[1])

np.save(f"eyetracking_x_data_Calib_{subject}_run_3", eyetracking_data_all_runs_x_intpol[2])
np.save(f"eyetracking_y_data_Calib_{subject}_run_3", eyetracking_data_all_runs_y_intpol[2])

In [ ]:
# Generate Expected Position 
# extract fixation point position 
fixations_positions = [
    [-9.0, 9.0], [-4.5, 9.0], [0.0, 9.0], [4.5, 9.0], [9.0, 9.0],
    [-9.0, 4.5], [-4.5, 4.5], [0.0, 4.5], [4.5, 4.5], [9.0, 4.5],
    [-9.0, 0.0], [-4.5, 0.0], [0.0, 0.0], [4.5, 0.0], [9.0, 0.0],
    [-9.0, -4.5],[-4.5, -4.5],[0.0, -4.5],[4.5, -4.5],[9.0, -4.5],
    [-9.0, -9.0],[-4.5, -9.0],[0.0, -9.0],[4.5, -9.0],[9.0, -9.0]
]



all_runs_expected_fix_x = []
all_runs_expected_fix_y = []

# save duration of trials in ms as well 
trial_durations_all_runs = []

for df_run, eye_data in zip(dfs_runs,eye_data_all_runs): 
    expected_position_fix = []

    for i in range(0,num_trials): 
        if not pd.isna(df_run['fixation_location'][i]):
            index = int(df_run['fixation_location'][i])-1
            expected_position_fix.append(fixations_positions[index])

        else: 
            expected_position_fix.append(np.nan)

    expected_position_fix_x = [item[0] if isinstance(item, list) else item for item in expected_position_fix]
    
    expected_position_fix_y = [item[1] if isinstance(item, list) else item for item in expected_position_fix]
    


    # check expected fixation positions 
    fig = px.scatter(x=expected_position_fix_x[:50], y=expected_position_fix_y[:50], width=800, height=800, template= "simple_white")
    fig.update_xaxes(range=[-10,10])
    fig.update_yaxes(range=[-10,10])

    #fig.show()


    # Align trial based expected position with milisecond format of eyetracking data

    trial_durations = df_run['duration']*1000
    run_durations = np.cumsum(trial_durations)
    trial_durations_all_runs.append(list(run_durations))

    # X direction
    trial_indices_x = np.searchsorted(run_durations, np.arange(len(eye_data[:,0]))).astype(int)
    
    expected_target_positions_x = np.array(expected_position_fix_x)
    trial_indices_x = np.clip(trial_indices_x, 0, len(expected_target_positions_x) - 1)
    expected_position_fix_x_aligned = expected_target_positions_x[trial_indices_x]
    all_runs_expected_fix_x.append(expected_position_fix_x_aligned)

    # Y direction
    trial_indices_y = np.searchsorted(run_durations, np.arange(len(eye_data[:,0]))).astype(int)
    
    expected_target_positions_y = np.array(expected_position_fix_y)
    trial_indices_y = np.clip(trial_indices_y, 0, len(expected_target_positions_y) - 1)
    expected_position_fix_y_aligned = expected_target_positions_y[trial_indices_y]
    all_runs_expected_fix_y.append(expected_position_fix_y_aligned)


In [ ]:
# Generate Expected Position 
# extract pursuit point position 
def interp1d(array: np.ndarray, new_len: int) -> np.ndarray:
    la = len(array)
    return np.interp(np.linspace(0, la - 1, num=new_len), np.arange(la), array)

import matplotlib.pyplot as plt
    
# Constants
TR_sec = 1.2
pursuit_dur_TR = 1
frame_duration = matfile['config']['scr'][0,0]['frame_duration'][0][0][0][0]
pursuit_dur_sec = matfile['config']['const'][0,0]['pursuit_dur_sec'][0][0][0][0]
pursuit_dur_frm = matfile['config']['const'][0,0]['pursuit_dur_frm'][0][0][0][0]
nb_trials_pursuit = matfile['config']['const'][0,0]['nb_trials_pursuit'][0][0][0][0]

amplitude = 'pursuit_amplitude'
angle = 'pursuit_angle'

all_runs_expected_purs_x = []
all_runs_expected_purs_y = []

pursuit_coord_on = np.array([float('nan'), float('nan')])
pursuit_coord_off = np.array([float('nan'), float('nan')])

#dfs_runs = [df_run_2]
#eye_data_all_runs = [eye_data_run_2]

for df_run, eye_data in zip(dfs_runs, eye_data_all_runs): 
    

    pursuit_coord_on_list = []
    pursuit_coord_off_list = []

    # Extracting the values from the DataFrame columns
    pursuit_amp_values = df_run[amplitude].values
    pursuit_angle_values = df_run[angle].values

    # map angle values
    legend_ang = {1: 0, 2: 20, 3: 40, 4: 60, 5: 80, 6: 100, 
            7: 120, 8: 140, 9: 160, 10: 180, 11: 200, 
            12: 220, 13: 240, 14: 260, 15: 280, 16: 300, 
            17: 320, 18: 340}

    # map amplitude values
    legend_amp = {1: 3, 2: 5, 3: 7}


    pursuit_angle_values_transf = [legend_ang[val] if not math.isnan(val) else float('nan') for val in pursuit_angle_values]
    pursuit_amp_values_transf = [legend_amp[val] if not math.isnan(val) else float('nan') for val in pursuit_amp_values]

    # covert amplitude dva to pixels
    pursuit_amp_values_transf = list(map(deg2pix, pursuit_amp_values_transf))
    

    for index, row in df_run.iterrows():

        if index == 52:  # first one should be in center
            pursuit_coord_on = np.array([960, 540])
            pursuit_coord_off = pursuit_coord_on + np.array([
                pursuit_amp_values_transf[index] * np.cos(np.radians(pursuit_angle_values_transf[index])),
                pursuit_amp_values_transf[index] * - np.sin(np.radians(pursuit_angle_values_transf[index]))])
            
            
        elif index == 105:
            pursuit_coord_on = pursuit_coord_off
            pursuit_coord_off = np.array([960, 540])  # final one should be in center

            
        else:
            pursuit_coord_on = pursuit_coord_off
            pursuit_coord_off = pursuit_coord_on + np.array([
                pursuit_amp_values_transf[index] * np.cos(np.radians(pursuit_angle_values_transf[index])),
                pursuit_amp_values_transf[index] * - np.sin(np.radians(pursuit_angle_values_transf[index]))])
            
           
        
        # Append to lists
        pursuit_coord_on_list.append(pursuit_coord_on)
        pursuit_coord_off_list.append(pursuit_coord_off)

        # Convert lists to NumPy arrays
        pursuit_coord_on_arr = np.array(pursuit_coord_on_list)
        pursuit_coord_off_arr = np.array(pursuit_coord_off_list)
    
    

    # Interpolate only the trials with pursuit coords in it into length of eyetracking data pursuit trials
    purs_x_intpl = interp1d(pursuit_coord_on_arr[51:107,0], new_len=len(eye_data[72000:138000]))
    purs_y_intpl = interp1d(pursuit_coord_on_arr[51:107,1], new_len=len(eye_data[72000:138000]))

    # Center and convert purs coordinates to dva

    purs_x_intpl_dva = (purs_x_intpl - (screen_size[0]/2))/ppd
    purs_y_intpl_dva = -1.0*((purs_y_intpl - (screen_size[1]/2))/ppd)

    purs_x_intpl_dva_na = purs_x_intpl_dva[~np.isnan(purs_x_intpl_dva)]
    all_runs_expected_purs_x.append(purs_x_intpl_dva_na)
    purs_y_intpl_dva_na = purs_y_intpl_dva[~np.isnan(purs_y_intpl_dva)]
    all_runs_expected_purs_y.append(purs_y_intpl_dva_na)

In [ ]:
# combine pursuit and fixation traces 
expected_x_run_1 = np.concatenate((np.array(all_runs_expected_fix_x[0][:int(trial_durations_all_runs[0][51])]), all_runs_expected_purs_x[0]))
expected_y_run_1 = np.concatenate((np.array(all_runs_expected_fix_y[0][:int(trial_durations_all_runs[0][51])]), all_runs_expected_purs_y[0]))

expected_x_run_2 = np.concatenate((np.array(all_runs_expected_fix_x[1][:int(trial_durations_all_runs[0][51])]), all_runs_expected_purs_x[1]))
expected_y_run_2 = np.concatenate((np.array(all_runs_expected_fix_y[1][:int(trial_durations_all_runs[0][51])]), all_runs_expected_purs_y[1]))

expected_x_run_3 = np.concatenate((np.array(all_runs_expected_fix_x[2][:int(trial_durations_all_runs[0][51])]), all_runs_expected_purs_x[2]))
expected_y_run_3 = np.concatenate((np.array(all_runs_expected_fix_y[2][:int(trial_durations_all_runs[0][51])]), all_runs_expected_purs_y[2]))

In [ ]:
expected_x_all_runs = [expected_x_run_1,expected_x_run_2,expected_x_run_3]
expected_y_all_runs = [expected_y_run_1,expected_y_run_2,expected_y_run_3]

In [ ]:
np.save(f'expected_x_calib_{subject}',expected_x_all_runs)
np.save(f'expected_y_calib_{subject}',expected_y_all_runs)

In [ ]:
import plotly.graph_objects as gosss

# Sample data
actual_data = x_dva_run_1[instruction_times[0]:instruction_times[0]+ int(trial_durations_all_runs[0][51])] 
expected_curve_fix = all_runs_expected_fix_x[0][:int(trial_durations_all_runs[0][51])]

# Create a trace for the actual data
trace_actual = go.Scatter(
    x=list(range(1, len(actual_data)+1)),
    y=actual_data,
    mode='lines',
    name='Eyetracker Gaze Position',
    line=dict(color='black', width=2.5),
)
#Create a trace for the expected curve
trace_expected_fix = go.Scatter(
    x=list(range(1, len(expected_curve_fix)+1)),
    y=expected_curve_fix,
    mode='lines',
    name='Expected Gaze Position',
    line=dict(color='lightskyblue', width=2.5, dash='dashdot'),
)

# Create layout with specific style for a nature paper
layout = go.Layout(
    title='Fixation task',
    title_x = 0.5,
    xaxis=dict(title='Time (ms)'),
    yaxis=dict(title='X Gaze Position', range = [-15,15]),
    showlegend=True,
    legend=dict(x=1, y=1),
    font=dict(family='Arial', size=12, color='black'),
    paper_bgcolor='white',  # Background color
    plot_bgcolor='white',   # Plot area background color
    margin=dict(l=50, r=50, t=50, b=50),  # Margins
)

# Create figure
fig = go.Figure(data=[trace_actual,trace_expected_fix], layout=layout)

# Show the plot

#fig.show()
pio.write_image(fig, 'pursuit_X_zoomed.jpeg',scale=6, width=2248, height=450)



In [ ]:
import plotly.graph_objects as gosss

# Sample data
actual_data = x_dva_run_2[instruction_times[1] + int(trial_durations_all_runs[0][51]):int(trial_durations_all_runs[0][107])] 
#expected_curve_fix = expected_position_fix_x_aligned
expected_curv_purs = all_runs_expected_purs_x[1]
#x_dva_purs_exp

# Create a trace for the actual data
trace_actual = go.Scatter(
    x=list(range(1, len(actual_data)+1)),
    y=actual_data,
    mode='lines',
    name='Eyetracker Gaze Position',
    line=dict(color='black', width=2.5),
)


#Create a trace for the expected curve
trace_expected_purs = go.Scatter(
    x=list(range(1, len(expected_curv_purs)+1)),
    y=expected_curv_purs,
    name='Expected Gaze Position',
    mode='lines',
    line=dict(color='lightskyblue', width=2.5, dash='dashdot'),
)

# Create layout with specific style for a nature paper
layout = go.Layout(
    title='Pursuit task',
    title_x = 0.5,
    xaxis=dict(title='Time (ms)'),
    yaxis=dict(title='X Gaze Position', range = [-15,15]),
    showlegend=True,
    legend=dict(x=1, y=1),
    font=dict(family='Arial', size=12, color='black'),
    paper_bgcolor='white',  # Background color
    plot_bgcolor='white',   # Plot area background color
    margin=dict(l=50, r=50, t=50, b=50),  # Margins
)

# Create figure
fig = go.Figure(data=[trace_actual,trace_expected_purs], layout=layout)

# Show the plot

#fig.show()

pio.write_image(fig, 'pursuit_X_zoomed_new.jpeg',scale=6, width=2248, height=450)



In [ ]:
deepmreye_sub_01_run_1_X = np.load("/Users/sinakling/projects/DeepMReye/calib_data/deepmreye_X_sub-01_run_01.npy")
deepmreye_sub_01_run_2_X = np.load("/Users/sinakling/projects/DeepMReye/calib_data/deepmreye_X_sub-01_run_02.npy")
deepmreye_sub_01_run_3_X = np.load("/Users/sinakling/projects/DeepMReye/calib_data/deepmreye_X_sub-01_run_03.npy")

deepmreye_data_all_runs_x = [deepmreye_sub_01_run_1_X,deepmreye_sub_01_run_2_X,deepmreye_sub_01_run_3_X]

deepmreye_sub_01_run_1_Y= np.load("/Users/sinakling/projects/DeepMReye/calib_data/deepmreye_Y_sub-01_run_01.npy")
deepmreye_sub_01_run_2_Y = np.load("/Users/sinakling/projects/DeepMReye/calib_data/deepmreye_Y_sub-01_run_02.npy")
deepmreye_sub_01_run_3_Y = np.load("/Users/sinakling/projects/DeepMReye/calib_data/deepmreye_Y_sub-01_run_03.npy")

deepmreye_data_all_runs_y = [deepmreye_sub_01_run_1_Y,deepmreye_sub_01_run_2_Y,deepmreye_sub_01_run_3_Y]

In [ ]:



import plotly.graph_objects as go

avg_list = [avg_diff_y]
expected_position = [expected_position_fix_x[0]]

for eyetracking_data, expected_position in zip(avg_list,expected_x_all_runs):

    # Sample data
    actual_data = eyetracking_data
    expected_curve = expected_position


    #x_dva_purs_exp

    # Create a trace for the actual data
    trace_actual = go.Scatter(
        x=list(range(1, len(actual_data)+1)),
        y=actual_data,
        mode='lines',
        name='Eyetracker Gaze Position',
        line=dict(color='#CD5334', width=2.5),
    )

# black: #0E1C36
    #Create a trace for the expected curve
    trace_expected = go.Scatter(
        x=list(range(1, len(expected_curve)+1)),
        y=expected_curve,
        name='Expected Gaze Position',
        mode='lines',
        line=dict(color='#45B69C', width=2.5, dash='dashdot'),
    )


    # Create layout with specific style for a nature paper
    layout = go.Layout(
        template="simple_white",
        title=f'{subject} run',
        title_x = 0.5,
        xaxis=dict(title='Time (sec)', tickmode = 'array',
            tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000],
            ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
        yaxis=dict(title='Vert. coord. (dva)', range = [-20,20]),
        showlegend=True,
        legend=dict(x=1, y=1),
        font=dict(family='Arial', size=12, color='black'),
        paper_bgcolor='white',  # Background color
        plot_bgcolor='white',   # Plot area background color
        margin=dict(l=50, r=50, t=50, b=50),  # Margins
    )

    # Create figure
    fig = go.Figure(data=[trace_actual], layout=layout)


    fig.add_vrect(x0="5400", x1="66500", 
                label=dict(
                text="fixation",
                textposition="top center"),
                fillcolor="#D8DDEF", opacity=0.2, line_width=0)

    fig.add_vrect(x0="71000", x1="137000", 
                label=dict(
                text="pursuit",
                textposition="top center"),
                fillcolor="#A0A4B8", opacity=0.2, line_width=0)

    fig.add_vrect(x0="141000", x1="179000", 
                label=dict(
                text="freeview",
                textposition="top center"),
                fillcolor="#7293A0", opacity=0.1, line_width=0)


    # Show the plot

    fig.show()

#CD5334


pio.write_image(fig, 'run_1_X.jpeg',scale=6, width=2248, height=450)



In [ ]:
import plotly.graph_objects as go
import numpy as np

# Animated plot

N = len(eyetracking_data_all_runs_x_intpol[2][::100])
x = eyetracking_data_all_runs_x_intpol[2][::100]
y = eyetracking_data_all_runs_y_intpol[2][::100]

# Create figure
fig = go.Figure(
    data=[
        go.Scatter(x=x[:1], y=y[:1], mode="lines", line=dict(width=2, color="black")),
    ],
    layout=go.Layout(
        xaxis=dict(range=[-15, 15], autorange=False, zeroline=False),
        yaxis=dict(range=[-15, 15], autorange=False, zeroline=False),
        title_text="Calibration task", hovermode="closest",
        updatemenus=[
            dict(
                type="buttons",
                buttons=[dict(label="Play", method="animate", args=[None])]
            )
        ]
    ),
    frames=[
        go.Frame(
            data=[
                go.Scatter(x=x[:k+1], y=y[:k+1], mode="lines", line=dict(width=2, color="black")),
            ],
            name=f"frame{k}"
        )
        for k in range(1, N)
    ]
)

fig.update_layout(
    template='simple_white',
    autosize=False,
    width=800,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100
    )
)

fig.show()


In [ ]:
# Subplot 

# create dataframe 
# remove instructions time from beginning and end to make all same length
def trim_lists_to_shortest(input_lists):
    # Find the length of the shortest list
    min_length = min(len(lst) for lst in input_lists)

    # Trim each list to the length of the shortest one
    trimmed_lists = [lst[:min_length] for lst in input_lists]

    return trimmed_lists

eyetracking_data_all_runs_x_trim = trim_lists_to_shortest(eyetracking_data_all_runs_x_intpol)
eyetracking_data_all_runs_y_trim = trim_lists_to_shortest(eyetracking_data_all_runs_y_intpol)


data = {"run_01_x": list(eyetracking_data_all_runs_x_trim[0]), "run_01_y": list(eyetracking_data_all_runs_y_trim[0]), "run_02_x": list(eyetracking_data_all_runs_x_trim[1]), "run_02_y": list(eyetracking_data_all_runs_y_trim[1]), "run_03_x": list(eyetracking_data_all_runs_x_trim[2]), "run_03_y": list(eyetracking_data_all_runs_y_trim[2])}

df = pd.DataFrame(data)
df

In [ ]:
deepmreye_data_all_runs_x_trim = trim_lists_to_shortest(deepmreye_data_all_runs_x)
deepmreye_data_all_runs_y_trim = trim_lists_to_shortest(deepmreye_data_all_runs_y)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



plot_rows = 3
plot_cols = 2

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=True,vertical_spacing=0.05, subplot_titles= ['Hor. Coord. run 1', 'Ver. Coord. run 1', 'Hor. Coord. run 2', 'Ver. Coord. run 2', 'Hor. Coord. run 3', 'Ver. Coord. run 3'])

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed

# add traces
x = 0
for i in range(1, plot_rows + 1):
    for j in range(1, plot_cols + 1):
        trace = go.Scatter(x=df.index, y=df[df.columns[x]].values,
                           name="Eyetracker Gaze Position",
                           mode='lines',
                           showlegend = True,
                           line=dict(color='#069D6B', width=1.5))

        fig.add_trace(trace, row=i, col=j)

        
        # Hide legend for each subplot
        #fig.update_traces(showlegend=True, selector=dict(name= "Eyetracker Gaze Position"))

        # Set common Y-axis range
        fig.update_yaxes(range=common_y_range, row=i, col=j)

        x += 1

fig.add_trace(go.Scatter(y= expected_x_all_runs[0],showlegend=True, name='Expected Gaze Position',line=dict(color='#0E1C36', width=2)), row = 1, col = 1)
fig.add_trace(go.Scatter(y= expected_y_all_runs[0],showlegend=False, line=dict(color='#0E1C36', width=2)), row = 1, col = 2)
fig.add_trace(go.Scatter(y= expected_x_all_runs[1],showlegend=False,line=dict(color='#0E1C36', width=2)), row = 2, col = 1)
fig.add_trace(go.Scatter(y= expected_y_all_runs[1],showlegend=False,line=dict(color='#0E1C36', width=2)), row = 2, col = 2)
fig.add_trace(go.Scatter(y= expected_x_all_runs[2],showlegend=False,line=dict(color='#0E1C36', width=2)), row = 3, col = 1)
fig.add_trace(go.Scatter(y= expected_y_all_runs[2],showlegend=False,line=dict(color='#0E1C36', width=2)), row = 3, col = 2)

fig.add_vrect(x0="5400", x1="66500", 
                label=dict(
                text="fixation",
                textposition="top center"),
                fillcolor="#D8DDEF", opacity=0.2, line_width=0)

fig.add_vrect(x0="71000", x1="137000", 
                label=dict(
                text="pursuit",
                textposition="top center"),
                fillcolor="#A0A4B8", opacity=0.2, line_width=0)

fig.add_vrect(x0="141000", x1="179000", 
                label=dict(
                text="freeview",
                textposition="top center"),
                fillcolor="#7293A0", opacity=0.1, line_width=0)

# Update the legend to be not visible for specifc traces
fig.update_traces(showlegend= True, row = 1, col = 1, selector=dict(name= "Scanner Eyetracker Gaze Position"))
fig.update_traces(showlegend=False, row=1, col=2)
fig.update_traces(showlegend=False, row=2, col=1)
fig.update_traces(showlegend=False, row=2, col=2)
fig.update_traces(showlegend=False, row=3, col=1)
fig.update_traces(showlegend=False, row=3, col=2)



# Format and show fig
fig.update_layout(height=1200, width=2000, template="simple_white", title_text=f"{subject} Eyetracker Gaze Position (X,Y) vs. Expected Gaze Position", 
            yaxis1 = dict(title = "<b>Hor. coord. (dva)<b>", title_font=dict(size=12)),
            yaxis2 = dict(title = "<b>Ver. coord. (dva)<b>",title_font=dict(size=12)),
            yaxis3 = dict(title = "<b>Hor. coord. (dva)<b>",title_font=dict(size=12)),
            yaxis4 = dict(title = "<b>Ver. coord. (dva)<b>",title_font=dict(size=12)),
            xaxis5=dict(title='<b>Time (sec)<b>', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis5 = dict(title = "<b>Hor. coord. (dva)<b>", title_font=dict(size=12)),
            xaxis6=dict(title='<b>Time (sec)<b>', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis6 = dict(title = "<b>Ver. coord. (dva)<b>"))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))


fig.show()
#fig.write_image(f'{subject}_scanner_gazeposition_eyetracker.pdf')



In [ ]:
# Average runs based on image id trials

# find trials depending on image id
image_ids_to_find = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
selected_trials_list = []

for df_run in dfs_runs:
    df_trials = [] 
    for image_id in image_ids_to_find:
        
        selected_trials = df_run.loc[df_run['image_id'] == image_id, 'trial_number'].tolist()
        df_trials.append(selected_trials)
    
    selected_trials_list.append(df_trials)


# slice eyetracking data for selected trials
all_runs_data = []

for run in range(len(dfs_runs)):
    run_data = []  
    
    for index in range(0, 10):
        im_x = deepmreye_data_all_runs_x[run][int(trial_durations_all_runs[0][selected_trials_list[run][index][0]-2]): int(trial_durations_all_runs[run][selected_trials_list[run][index][0]-1])]
        im_y = deepmreye_data_all_runs_y[run][int(trial_durations_all_runs[0][selected_trials_list[run][index][0]-2]): int(trial_durations_all_runs[run][selected_trials_list[run][index][0]-1])]
        
        run_data.append([im_x, im_y]) 
    
    all_runs_data.append(run_data)  


# Average image trials over runs (x)
image_avg_x = []


min_lengths = [min(len(all_runs_data[0][index][0]), len(all_runs_data[1][index][0]), len(all_runs_data[2][index][0])) for index in range(0,10)]
for index in range(10):
    avg = (all_runs_data[0][index][0][:min_lengths[index]] +
            all_runs_data[1][index][0][:min_lengths[index]] +
            all_runs_data[2][index][0][:min_lengths[index]]) / 3
    image_avg_x.append(avg)


# Average image trials over runs (y)
image_avg_y = []


    
min_lengths = [min(len(all_runs_data[0][index][1]), len(all_runs_data[1][index][1]), len(all_runs_data[2][index][1])) for index in range(0,10)]
for index in range(10):
    avg = (all_runs_data[0][index][1][:min_lengths[index]] +
            all_runs_data[1][index][1][:min_lengths[index]] +
            all_runs_data[2][index][1][:min_lengths[index]]) / 3
    image_avg_y.append(avg)


In [ ]:
# Plot average gaze position 


fig = make_subplots(rows=2, cols=5, subplot_titles=[f"Image {i}" for i in range(1, 11)], shared_xaxes=True, shared_yaxes=True)

# Loop through each subplot
for index in range(1, 11):
    if index == 10:
        image_source = f"/Users/sinakling/projects/deepmreyecalib/experiment_code/stim/images/image_{index}.png"
    else:
        image_source = f"/Users/sinakling/projects/deepmreyecalib/experiment_code/stim/images/image_0{index}.png"

    trace_data = go.Scatter(
        x=image_avg_x[index-1],  # Adjust index to start from 0
        y=image_avg_y[index-1],  # Adjust index to start from 0
        showlegend=False,
        name='freeviewing',
        mode='markers',
        marker=dict(color='black'))

    fig.add_trace(trace_data, row=(index-1)//5 + 1, col=(index-1)%5 + 1)

    fig.add_layout_image(
        dict(
            source=image_source,
            xref=f"x{index}",
            yref=f"y{index}",
            x=-9,
            y=9,
            sizex=18,
            sizey=18,
            opacity=0.9,
            layer="below"
        ))

    fig.update_xaxes(range=[-10, 10], row=(index-1)//5 + 1, col=(index-1)%5 + 1)
    fig.update_yaxes(range=[-10, 10], row=(index-1)//5 + 1, col=(index-1)%5 + 1)

# Update layout
fig.update_layout(
    template='simple_white',
    autosize=False,
    title_text=f"{subject} Average Gaze Position",
    width=1600,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100
    )
)

# Show the figure
fig.show()
#fig.write_image(f'{subject}_deepmreye_avg_gaze_position_images.pdf')


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming df_run_1 is your dataframe
rows_to_plot = range(107, 117)

fig = make_subplots(rows=len(rows_to_plot), cols=1, vertical_spacing=0.01)

for i, row in enumerate(rows_to_plot, 1):
    image_id = int(df_run_1['image_id'][row])
    
    if image_id == 10:
        image_source = f"/Users/sinakling/projects/deepmreyecalib/stim/images/image_{image_id}.png"
    else:
        image_source = f"/Users/sinakling/projects/deepmreyecalib/stim/images/image_0{image_id}.png"

    fig.add_trace(
        go.Scatter(
            x=x_dva_run_1[int(trial_durations_all_runs[0][row-1]):int(trial_durations_all_runs[0][row])],
            y=y_dva_run_1[int(trial_durations_all_runs[0][row-1]):int(trial_durations_all_runs[0][row])],
            showlegend=False,
            name='freeviewing',
            mode='markers',
            marker=dict(color='black'),
            opacity=0.8
        ),
        row=i, col=1
    )

    fig.add_layout_image(
        dict(
            source=image_source,
            xref="x",
            yref="y",
            x=-9,
            y=9,
            sizex=18,
            sizey=18,
            opacity=0.9,
            layer="below"
        ),
        row=i, col=1
    )

    fig.update_xaxes(range=[-15, 15], row=i, col=1)
    fig.update_yaxes(range=[-15, 15], row=i, col=1)

fig.update_layout(
    template='simple_white',
    autosize=False,
    width=500,
    height=500 * len(rows_to_plot),
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100
    )
)

fig.show()
fig.write_image(f'{subject}_run_1_freeviewing.pdf')


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

x_dva_all_runs = [x_dva_run_1, x_dva_run_2, x_dva_run_3]
y_dva_all_runs = [y_dva_run_1, y_dva_run_2, y_dva_run_3]
# Assuming df_run_1 is your dataframe
rows_to_plot = range(107, 117)



for df_run, data_x, data_y in zip(dfs_runs, x_dva_all_runs, y_dva_all_runs):

    fig = make_subplots(rows=len(rows_to_plot), cols=1, vertical_spacing=0.01)

    for i, row in enumerate(rows_to_plot, 1):
        image_id = int(df_run['image_id'][row])

        if image_id == 10:
            image_source = f"/Users/sinakling/projects/deepmreyecalib/stim/images/image_{image_id}.png"
        else:
            image_source = f"/Users/sinakling/projects/deepmreyecalib/stim/images/image_0{image_id}.png"

        x_data = data_x[int(trial_durations_all_runs[0][row]):int(trial_durations_all_runs[0][row + 1])]
        y_data = data_y[int(trial_durations_all_runs[0][row]):int(trial_durations_all_runs[0][row + 1])]

        # Add freeviewing scatter plot
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=y_data,
                showlegend=False,
                name='freeviewing',
                mode='markers',
                marker=dict(color='black'),
                opacity=0.8
            ),
            row=i, col=1
        )

        # Add density heatmap
        fig.add_trace(go.Histogram2d(
            x=x_data,
            y=y_data,
            showlegend=False,
            opacity= 0.7
        ),
            row=i, col=1
        )

        fig.add_layout_image(
        dict(
            source=image_source,
            xref="x",
            yref="y",
            x=-9,
            y=9,
            sizex=18,
            sizey=18,
            opacity=0.9,
            layer="below"
        ),
        row=i, col=1
    )
        
        fig.update_xaxes(range=[-15, 15], row=i, col=1)
        fig.update_yaxes(range=[-15, 15], row=i, col=1)

    fig.update_layout(
        template='simple_white',
        autosize=False,
        width=500,
        height=500 * len(rows_to_plot),
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100
        )
    )

    fig.show()
    #fig.write_image(f'{subject}_hm_freeviewing.pdf')


       




In [ ]:
def euclidian_error(point_1, point_2):
    array_1, array_2 = np.array(point_1), np.array(point_2)
    squared_distance = np.sum(np.square(array_1 - array_2))
    distance = np.sqrt(squared_distance)
    return distance

In [ ]:
def pointwise_difference(array1, array2):
    """
    Calculate the pointwise difference between two arrays.

    Parameters:
    - array1: First input array
    - array2: Second input array

    Returns:
    - result: Array containing the pointwise differences
    """
    

    result = [a - b for a, b in zip(array1, array2)]
    return result


In [ ]:
#copy of data
eyetracking_data_all_runs_x_no_fv = eyetracking_data_all_runs_x_intpol

#only look at data of task 1 and 2 (leave out freeview)
eyetracking_data_all_runs_x_no_fv[0] = eyetracking_data_all_runs_x_no_fv[0][:len(expected_x_all_runs[0])]
eyetracking_data_all_runs_x_no_fv[1] = eyetracking_data_all_runs_x_no_fv[1][:len(expected_x_all_runs[1])]
eyetracking_data_all_runs_x_no_fv[2] = eyetracking_data_all_runs_x_no_fv[2][:len(expected_x_all_runs[2])]

eyetracking_data_all_runs_y_no_fv = eyetracking_data_all_runs_y_intpol

eyetracking_data_all_runs_y_no_fv[0] = eyetracking_data_all_runs_y_no_fv[0][:len(expected_y_all_runs[0])]
eyetracking_data_all_runs_y_no_fv[1] = eyetracking_data_all_runs_y_no_fv[1][:len(expected_y_all_runs[1])]
eyetracking_data_all_runs_y_no_fv[2] = eyetracking_data_all_runs_y_no_fv[2][:len(expected_y_all_runs[2])]

In [ ]:
# Calculate EE for horizontal coords
error_x_all_runs = []

for run in range(len(eyetracking_data_all_runs_x_no_fv)): 
    error_x= []
    
    if len(eyetracking_data_all_runs_x_no_fv[run]) != len(expected_x_all_runs[run]):
        print(f"Error: Length mismatch in run {run}")
        
        continue

    for i in range(len(eyetracking_data_all_runs_x_no_fv[run])):
        
        if i < len(eyetracking_data_all_runs_x_no_fv[run]) and i < len(expected_x_all_runs[run]):
            error_x.append(euclidian_error(eyetracking_data_all_runs_x[run][i], expected_x_all_runs[run][i]))
        else:
            print(f"Error: Index out of bounds in run {run}, index {i}")
                
    error_x_all_runs.append(error_x)

In [ ]:
# Calculate EE for horizontal coords
error_x_all_runs = []

for run in range(len(eyetracking_data_all_runs_x_trim)): 
    error_x= []
    
    if len(eyetracking_data_all_runs_x_trim[run]) != len(deepmreye_data_all_runs_x_trim[run]):
        print(f"Error: Length mismatch in run {run}")
        
        continue

    for i in range(len(eyetracking_data_all_runs_x_trim[run])):
        
        if i < len(eyetracking_data_all_runs_x_trim[run]) and i < len(deepmreye_data_all_runs_x_trim[run]):
            error_x.append(euclidian_error(eyetracking_data_all_runs_x_trim[run][i], deepmreye_data_all_runs_x_trim[run][i]))
        else:
            print(f"Error: Index out of bounds in run {run}, index {i}")
                
    error_x_all_runs.append(error_x)

In [ ]:
#Calculate EE for vertical coords
error_y_all_runs = []

for run in range(len(eyetracking_data_all_runs_y_no_fv)): 
    error_y= []
    
    if len(eyetracking_data_all_runs_y_no_fv[run]) != len(expected_y_all_runs[run]):
        print(f"Error: Length mismatch in run {run}")
        
        continue

    for i in range(len(eyetracking_data_all_runs_y_no_fv[run])):
        
        if i < len(eyetracking_data_all_runs_y_no_fv[run]) and i < len(expected_y_all_runs[run]):
            error_y.append(euclidian_error(eyetracking_data_all_runs_y_no_fv[run][i], expected_y_all_runs[run][i]))
        else:
            print(f"Error: Index out of bounds in run {run}, index {i}")
                
    error_y_all_runs.append(error_y)

In [ ]:
# Calculate EE for horizontal coords
error_y_all_runs = []

for run in range(len(eyetracking_data_all_runs_y_trim)): 
    error_y= []
    
    if len(eyetracking_data_all_runs_y_trim[run]) != len(deepmreye_data_all_runs_y_trim[run]):
        print(f"Error: Length mismatch in run {run}")
        
        continue

    for i in range(len(eyetracking_data_all_runs_y_trim[run])):
        
        if i < len(eyetracking_data_all_runs_y_trim[run]) and i < len(deepmreye_data_all_runs_y_trim[run]):
            error_y.append(euclidian_error(eyetracking_data_all_runs_y_trim[run][i], deepmreye_data_all_runs_y_trim[run][i]))
        else:
            print(f"Error: Index out of bounds in run {run}, index {i}")
                
    error_y_all_runs.append(error_y)

In [ ]:
# Plot Euclidan Error in Subplot

import plotly.graph_objects as go
from plotly.subplots import make_subplots



plot_rows = 3
plot_cols = 2

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=True,vertical_spacing=0.05)

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed


fig.add_trace(go.Scatter(y= error_x_all_runs[0],showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 1, col = 1)
fig.add_trace(go.Scatter(y= error_y_all_runs[0],showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 1, col = 2)
fig.add_trace(go.Scatter(y= error_x_all_runs[1],showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 2, col = 1)
fig.add_trace(go.Scatter(y= error_y_all_runs[1],showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 2, col = 2)
fig.add_trace(go.Scatter(y= error_x_all_runs[2],showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 3, col = 1)
fig.add_trace(go.Scatter(y= error_y_all_runs[2],showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 3, col = 2)

fig.add_vrect(x0="5400", x1="66500", 
                label=dict(
                text="fixation",
                textposition="top center"),
                fillcolor="#D8DDEF", opacity=0.2, line_width=0)

fig.add_vrect(x0="72000", x1="136000", 
                label=dict(
                text="pursuit",
                textposition="top center"),
                fillcolor="#A0A4B8", opacity=0.2, line_width=0)



# Format and show fig
fig.update_layout(height=1200, width=2000, template="simple_white", title_text=f"{subject} Eyetracking Euclidean Error", 
            yaxis1 = dict(title = "EE"),
            yaxis2 = dict(title = "EE"),
            yaxis3 = dict(title = "EE"),
            yaxis4 = dict(title = "EE"),
            xaxis5=dict(title='Time (sec)', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis5 = dict(title = "EE"),
            xaxis6=dict(title='Time (sec)', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis6 = dict(title = "EE"))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))

fig.show()
#fig.write_image(f'{subject}_eyetracking_error.pdf')



In [ ]:
# Simple difference of curves (in dva) 

difference_x_run_1 = pointwise_difference(eyetracking_data_all_runs_x_no_fv[0],expected_x_all_runs[0])
difference_x_run_2 = pointwise_difference(eyetracking_data_all_runs_x_no_fv[1],expected_x_all_runs[1])
difference_x_run_3 = pointwise_difference(eyetracking_data_all_runs_x_no_fv[2],expected_x_all_runs[2])

In [ ]:
difference_y_run_1 = pointwise_difference(eyetracking_data_all_runs_y_no_fv[0],expected_y_all_runs[0])
difference_y_run_2 = pointwise_difference(eyetracking_data_all_runs_y_no_fv[1],expected_y_all_runs[1])
difference_y_run_3 = pointwise_difference(eyetracking_data_all_runs_y_no_fv[2],expected_y_all_runs[2])

In [ ]:
# difference of eye tracking and deepmreye

difference_x_run_1 = pointwise_difference(eyetracking_data_all_runs_x_trim[0],deepmreye_data_all_runs_x_trim[0])
difference_x_run_2 = pointwise_difference(eyetracking_data_all_runs_x_trim[1],deepmreye_data_all_runs_x_trim[1])
difference_x_run_3 = pointwise_difference(eyetracking_data_all_runs_x_trim[2],deepmreye_data_all_runs_x_trim[2])

In [ ]:
# difference of eye tracking and deepmreye

difference_y_run_1 = pointwise_difference(eyetracking_data_all_runs_y_trim[0],deepmreye_data_all_runs_y_trim[0])
difference_y_run_2 = pointwise_difference(eyetracking_data_all_runs_y_trim[1],deepmreye_data_all_runs_y_trim[1])
difference_y_run_3 = pointwise_difference(eyetracking_data_all_runs_y_trim[2],deepmreye_data_all_runs_y_trim[2])

In [ ]:
avg_diff_x = (np.array(difference_x_run_1) + np.array(difference_x_run_2) + np.array(difference_x_run_3)) / 3
avg_diff_y = (np.array(difference_y_run_1) + np.array(difference_y_run_2) + np.array(difference_y_run_3)) / 3

In [ ]:
# Plot Difference in Subplot

import plotly.graph_objects as go
from plotly.subplots import make_subplots



plot_rows = 3
plot_cols = 2

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=True,vertical_spacing=0.05)

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed


fig.add_trace(go.Scatter(y= difference_x_run_1,showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 1, col = 1)
fig.add_trace(go.Scatter(y= difference_y_run_1,showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 1, col = 2)
fig.add_trace(go.Scatter(y= difference_x_run_2,showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 2, col = 1)
fig.add_trace(go.Scatter(y= difference_y_run_2,showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 2, col = 2)
fig.add_trace(go.Scatter(y= difference_x_run_3,showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 3, col = 1)
fig.add_trace(go.Scatter(y= difference_y_run_3,showlegend=False,line=dict(color='#CD5334', width=2, dash = "solid")), row = 3, col = 2)

fig.add_vrect(x0="5400", x1="66500", 
                label=dict(
                text="fixation",
                textposition="top center"),
                fillcolor="#D8DDEF", opacity=0.2, line_width=0)

fig.add_vrect(x0="72000", x1="136000", 
                label=dict(
                text="pursuit",
                textposition="top center"),
                fillcolor="#A0A4B8", opacity=0.2, line_width=0)



# Format and show fig
fig.update_layout(height=1200, width=2000, template="simple_white", title_text=f"{subject} Scanner Eyetracking Position (X,Y) relative to Expected Position (X,Y)", 
            yaxis1 = dict(title = "difference in dva (X)"),
            yaxis2 = dict(title = "difference in dva (Y)"),
            yaxis3 = dict(title = "difference in dva (X)"),
            yaxis4 = dict(title = "difference in dva (Y)"),
            xaxis5=dict(title='Time (sec)', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis5 = dict(title = "difference in dva (X)"),
            xaxis6=dict(title='Time (sec)', tickmode = 'array', tickvals = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000], ticktext = ['0', '20', "40", "60","80", "100", "120", "140", "160", "180"]),
            yaxis6 = dict(title = "difference in dva (Y)"))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))

fig.show()
#fig.write_image(f'{subject}_scanner_eyetracking_difference.pdf')



In [ ]:
fig = go.Figure()


dif_x_1_fix = difference_x_run_1[6000:]
dif_x_2_fix = difference_x_run_2[6000:]
dif_x_3_fix = difference_x_run_3[6000:]
fig.add_trace(go.Box(y=dif_x_1_fix[::50], showlegend = False, marker_color='rgb(160, 164, 184)',
    line_color='rgb(1, 1, 1)', fillcolor='rgb(160, 164, 184)', name='fixation run 1'))
fig.add_trace(go.Box(y=dif_x_2_fix[::50], showlegend = False, marker_color='rgb(160, 164, 184)',
    line_color='rgb(1, 1, 1)',fillcolor='rgb(160, 164, 184)',name='fixation run 2'))
fig.add_trace(go.Box(y=dif_x_3_fix[::50], showlegend = False, marker_color='rgb(160, 164, 184)',
    line_color='rgb(1, 1, 1)',fillcolor='rgb(160, 164, 184)',name='fixation run 3'))

#fig.update_yaxes(range = [-30,17])                          
fig.update_layout(template="simple_white", title_text=f"{subject} Difference Scanner Eyetracker Gaze Position and Expected Position")
fig.write_image(f'{subject}_scanner_eyetracking_difference_fix_boxplot.pdf')

fig.show()

In [ ]:
fig = go.Figure()


dif_x_1_purs = difference_x_run_1[71000:]
dif_x_2_purs = difference_x_run_2[71000:]
dif_x_3_purs = difference_x_run_3[71000:]
fig.add_trace(go.Box(y=dif_x_1_purs[::50], showlegend = False, marker_color='rgb(160, 164, 184)',
    line_color='rgb(1, 1, 1)', fillcolor='rgb(160, 164, 184)', name = 'pursuit run 1'))
fig.add_trace(go.Box(y=dif_x_1_purs[::50], showlegend = False, marker_color='rgb(160, 164, 184)',
    line_color='rgb(1, 1, 1)',fillcolor='rgb(160, 164, 184)', name = 'pursuit run 2'))
fig.add_trace(go.Box(y=dif_x_1_purs[::50], showlegend = False, marker_color='rgb(160, 164, 184)',
    line_color='rgb(1, 1, 1)',fillcolor='rgb(160, 164, 184)', name = 'pursuit run 3'))

#fig.update_yaxes(range = [-10,10])                          
fig.update_layout(template="simple_white", title_text=f"{subject} Difference Scanner Eyetracker Gaze Position and Expected Position")

fig.write_image(f'{subject}_scanner_eyetracking_difference_purs_boxplot.pdf')
fig.show()

In [ ]:
# Average errors over runs 

difference_x_avg = (np.array(difference_x_run_1) + np.array(difference_x_run_2) + np.array(difference_x_run_3)) / 3
difference_y_avg = (np.array(difference_y_run_1) + np.array(difference_y_run_2) + np.array(difference_y_run_3)) / 3

In [ ]:
# Plot Difference Histogram in Subplot

import plotly.graph_objects as go
from plotly.subplots import make_subplots



plot_rows = 3
plot_cols = 2

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=True,vertical_spacing=0.05, subplot_titles=["Fixation run 1", "Pursuit run 1", "Fixation run 2", "Pursuit run 2", "Fixation run 3", "Pursuit run 3"])

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed


fig.add_trace(go.Histogram2d(x = difference_x_run_1[int(run_durations[0]):int(run_durations[51])],y = difference_y_run_1[int(run_durations[0]):int(run_durations[51])], colorscale='gray', reversescale= True, showscale = False), row = 1, col = 1)
fig.add_trace(go.Histogram2d(x = difference_x_run_2[int(run_durations[0]):int(run_durations[51])],y = difference_y_run_2[int(run_durations[0]):int(run_durations[51])], colorscale= 'gray', reversescale= True,showscale = False), row = 2, col = 1)
fig.add_trace(go.Histogram2d(x = difference_x_run_3[int(run_durations[0]):int(run_durations[51])],y = difference_y_run_3[int(run_durations[0]):int(run_durations[51])], colorscale= 'gray', reversescale= True,showscale = False), row = 3, col = 1)
fig.add_trace(go.Histogram2d(x = difference_x_run_1[int(run_durations[51]):],y = difference_y_run_1[int(run_durations[51]):], colorscale= 'gray', reversescale= True,showscale = False), row = 1, col = 2)
fig.add_trace(go.Histogram2d(x = difference_x_run_1[int(run_durations[51]):],y = difference_y_run_1[int(run_durations[51]):], colorscale= 'gray', reversescale= True,showscale = False), row = 2, col = 2)
fig.add_trace(go.Histogram2d(x = difference_x_run_1[int(run_durations[51]):],y = difference_y_run_1[int(run_durations[51]):], colorscale= 'gray', reversescale= True,showscale = False), row = 3, col = 2)



# Format and show fig
fig.update_layout(height=1200, width=2000, template="simple_white", title_text=f"{subject} Eyetracking relative to Expected Position", 
            yaxis1 = dict(title = "Vert. coord. (dva)", range = [-10,10]),
            yaxis2 = dict(title = "Vert. coord. (dva)",range = [-10,10]),
            yaxis3 = dict(title = "Vert. coord. (dva)",range = [-10,10]),
            yaxis4 = dict(title = "Vert. coord. (dva)",range = [-10,10]),
            xaxis5=dict(title='Hor. coord. (dva)',range = [-10,10]),
            yaxis5 = dict(title = "Vert. coord. (dva)",range = [-10,10]),
            xaxis6=dict(title='Hor. coord. (dva)',range = [-10,10]),
            yaxis6 = dict(title = "Vert. coord. (dva)",range = [-10,10]))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))

fig.update_layout(
    coloraxis_showscale=False,
    autosize=False,
    width=2000,
    height=2000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ))

fig.update_coloraxes(showscale=False)

fig.show()
fig.write_image(f'{subject}_eyetracking_relative_gaze.pdf')

#colorscale=[[1, 'rgb(255,255,255)'], [0.75, 'rgb(160,164,184)'], [0.5, 'rgb(169,198,184)'], [0.25, 'rgb(216,221,239)'], [0, 'rgb(0,0,0)']]



In [ ]:
# Plot Averge Difference Histogram in Subplot

import plotly.graph_objects as go
from plotly.subplots import make_subplots



plot_rows = 1
plot_cols = 3

fig = make_subplots(rows=plot_rows, cols=plot_cols,shared_xaxes=False, vertical_spacing=0.05, subplot_titles=["Fixation", "Pursuit", "Fixation + Pursuit"])

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed


fig.add_trace(go.Histogram2d(x = difference_x_avg[int(run_durations[0]):int(run_durations[51])],y = difference_y_avg[int(run_durations[0]):int(run_durations[51])],  name = "Fixation difference", colorscale='gray', reversescale= True, showscale = False), row = 1, col = 1)
fig.add_trace(go.Histogram2d(x = difference_x_avg[int(run_durations[51]):],y = difference_y_avg[int(run_durations[51]):], colorscale= 'gray', reversescale= True,  name = "Pursuit difference", showscale = False), row = 1, col = 2)
fig.add_trace(go.Histogram2d(x = difference_x_avg[int(run_durations[0]):],y = difference_y_avg[int(run_durations[0]):], name = "All difference",colorscale= 'gray', reversescale= True,showscale = True), row = 1, col = 3)




# Format and show fig
fig.update_layout(height=1200, width=2000, template="simple_white", title_text=f"{subject}: Average histograms of pretrained deepmreye positions relative to expected positions (0, 0)",  #difference? 
            yaxis1 = dict(title = "<b>Ver. coord. (dva)<b>", range = [-5,5], title_font=dict(size=10)),
            xaxis1 = dict(title = "<b>Hor. coord. (dva)<b>", range = [-5,5], title_font=dict(size=10)),
            yaxis2 = dict(title = "<b>Ver. coord. (dva)<b>", range = [-5,5], title_font=dict(size=10)),
            xaxis2 = dict(title = "<b>Hor. coord. (dva)<b>", range = [-5,5], title_font=dict(size=10)),
            yaxis3 = dict(title = "<b>Ver. coord. (dva)<b>", range = [-5,5], title_font=dict(size=10)),
            xaxis3 = dict(title = "<b>Hor. coord. (dva)<b>", range = [-5,5], title_font=dict(size=10)))


# Update subplot titles font
fig.update_annotations(font=dict(size=14))

fig.update_layout(
    coloraxis_showscale=False,
    autosize=False,
    width=1600,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,


        t=100,
        pad=4
    ))

fig.update_coloraxes(showscale=False)

fig.show()
#fig.write_image(f'{subject}_scanner_eyetracking_relative_gaze_avg.pdf')

#colorscale=[[1, 'rgb(255,255,255)'], [0.75, 'rgb(160,164,184)'], [0.5, 'rgb(169,198,184)'], [0.25, 'rgb(216,221,239)'], [0, 'rgb(0,0,0)']]





In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np



# Define the number of rows and columns for the subplot
plot_rows = 1
plot_cols = 3

# Create subplots with shared x-axis
fig = make_subplots(rows=plot_rows, cols=plot_cols, shared_xaxes=False, vertical_spacing=0.05, subplot_titles=["Fixation", "Pursuit", "Fixation + Pursuit"])

# Set a common y-axis range
common_y_range = [-15, 15]  # Adjust the range as needed

# Add initial traces to subplots
fixation_hist = go.Histogram2d(x=[], y=[], name="Fixation difference", colorscale='gray', reversescale=True, showscale=False)
pursuit_hist = go.Histogram2d(x=[], y=[], name="Pursuit difference", colorscale='gray', reversescale=True, showscale=False)
all_hist = go.Histogram2d(x=[], y=[], name="All difference", colorscale='gray', reversescale=True, showscale=True)

fig.add_trace(fixation_hist, row=1, col=1)
fig.add_trace(pursuit_hist, row=1, col=2)
fig.add_trace(all_hist, row=1, col=3)

# Create frames
frames = []
for i in range(len(run_durations) - 1):
    fix_data = go.Histogram2d(x=difference_x_avg[int(run_durations[i]):int(run_durations[i + 1])], y=difference_y_avg[int(run_durations[i]):int(run_durations[i + 1])])
    pursuit_data = go.Histogram2d(x=difference_x_avg[int(run_durations[i + 1]):], y=difference_y_avg[int(run_durations[i + 1]):])
    all_data = go.Histogram2d(x=difference_x_avg[int(run_durations[i]):], y=difference_y_avg[int(run_durations[i]):])
    
    frame = go.Frame(data=[fix_data, pursuit_data, all_data], name=str(i))
    frames.append(frame)

# Update layout
fig.update_layout(
    height=1200, 
    width=2000, 
    template="simple_white", 
    title_text=f"{subject}: Average histograms of scanner eyetracker positions relative to expected positions (0, 0)",  #difference? 
    yaxis1=dict(title="<b>Ver. coord. (dva)<b>", range=[-5, 5], title_font=dict(size=10)),
    xaxis1=dict(title="<b>Hor. coord. (dva)<b>", range=[-5, 5], title_font=dict(size=10)),
    yaxis2=dict(title="<b>Ver. coord. (dva)<b>", range=[-5, 5], title_font=dict(size=10)),
    xaxis2=dict(title="<b>Hor. coord. (dva)<b>", range=[-5, 5], title_font=dict(size=10)),
    yaxis3=dict(title="<b>Ver. coord. (dva)<b>", range=[-5, 5], title_font=dict(size=10)),
    xaxis3=dict(title="<b>Hor. coord. (dva)<b>", range=[-5, 5], title_font=dict(size=10)),
    updatemenus=[dict(
        type="buttons",
        buttons=[dict(label="Play",
                      method="animate",
                      args=[None, {"frame": {"duration": 500}, "fromcurrent": True, "transition": {"duration": 300, "easing": "quadratic-in-out"}}])]
    )]
)

# Update subplot titles font
fig.update_annotations(font=dict(size=14))

fig.update_layout(
    coloraxis_showscale=False,
    autosize=False,
    width=1600,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    )
)

fig.update_coloraxes(showscale=False)

# Add frames to the figure
fig.frames = frames

# Display the animation
fig.show()
